In [3]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

#prmtop = 
#inpcrd = 
pdb = PDBFile('8B2T_nirmatrelvir_prepared.pdb') #add missing atoms (chapter 4)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml') #standard force fields (section 3.6.2)

print('Loading...')
modeller = Modeller(pdb.topology, pdb.positions) #this is now the new "modeled" pdb, note how pdb. is not called anymore

print('Adding hydrogens...') #add hydrogens
modeller.addHydrogens(forcefield)

print('Adding solvent...') #add solvent
modeller.addSolvent(forcefield,
                    ionicStrength=0.1*molar,
                    padding=1*nanometer,
                    )
print(modeller.getTopology())

print('Creating system...') #create system
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
                                 nonbondedCutoff=10*angstrom, constraints=HBonds)

print('Integrating') #integrator for constant temperature/energy simulations
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

print('Preparing simulaiton...')
simulation = Simulation(modeller.topology, system, integrator)

print('Setting initial positions...')
simulation.context.setPositions(modeller.positions,)

print('Conducting local minimization...')
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('8B2T_nirmatrelvir_minimized.pdb', 1000)) #generate output PDB every 1000 steps with reporter
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                                              potentialEnergy=True, temperature=True))
simulation.step(10000)
print('Done!')


/home/jsanlleyhernandez/miniconda3/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM    3  C   NRM A   0      -1.657 -13.520  42.636  1.00  0.00           C  , HETATM    2  C   NRM A   0      -0.203 -13.886  42.684  1.00  0.00           C  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/jsanlleyhernandez/miniconda3/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM    4  C   NRM A   0      -1.799 -12.029  42.706  1.00  0.00           C  , HETATM    3  C   NRM A   0      -1.657 -13.520  42.636  1.00  0.00           C  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/jsanlleyhernandez/miniconda3/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: 

Loading...
Adding hydrogens...


ValueError: No template found for residue 1 (NRM).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template